In [ ]:
'''
Created 01/02/2022
by Samuel Valman https://github.com/SamValman
for Templeton Bahamas Project, UoN Rights Lab
Data downloaded from GEE to Google Drive (templetonprojectbah@gmail.com) using batches derived from bahamas outline map. 
Full method in report.
'''

In [ ]:
import pandas as pd
import geemap
import ee
import os
ee.Initialize()
import time

In [ ]:
# def Planet_Cloud_Function(image):
#     mask = image.select('udm1').eq(0)
#     return image.updateMask(mask)


c = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.inList('ADM0_NAME',['Bahamas']))
batch = ee.FeatureCollection('users/templetonprojectbah/bahamas_small')

In [ ]:
batch_ls = batch.toList(400)
# batch.size().getInfo()

In [ ]:
#scale 
scale =  3.5

# use this one not the noNormal from here on in 

In [ ]:
def download_planet_rgbOnly(ic,shp, fn,folder, scale, step, clip=0, clip_ls=0):
    '''
    requires: geemap, ee, os
    step must be list
    path = real path
    fn = 'name.tif'  string
    '''
    def Planet_Cloud_Function(image):
        mask = image.select('udm1').eq(0)
        return image.updateMask(mask)
    ic = ic.map(lambda x:Planet_Cloud_Function(x))
    im = ic.median().clip(shp)
    
    
    # dowload
    if clip ==1:
        batch = clip_ls
        for i in range(0,clip_ls.size().getInfo()):
        
            shp = ee.Feature(batch.get(i))

            filen = fn + str(i)
            geemap.ee_export_image_to_drive(im, description = filen, folder=folder, scale=scale, region=shp.geometry())
            time.sleep(60)
    else:
        batch = ee.FeatureCollection('users/templetonprojectbah/bahamas_small').toList(400)    
    
        for i in step:
            shp = ee.Feature(batch.get(i))

            filen = fn + str(i)
            geemap.ee_export_image_to_drive(im, description = filen, folder=folder, scale=scale, region=shp.geometry())
            time.sleep(100)



In [ ]:
# import time
# time.sleep(43200)

In [ ]:
def download_planet_deriv(ic,shp, fn,folder, scale, step, clip=0, clip_ls=0):
    '''
    requires: geemap, ee, os
    step must be list
    path = real path
    fn = 'name.tif'  string
    '''
    def Planet_Cloud_Function(image):
        mask = image.select('udm1').eq(0)
        return image.updateMask(mask)
    ic = ic.map(lambda x:Planet_Cloud_Function(x))
    im = ic.median().clip(shp)
    
    # bandmath 
    ndvi = im.normalizedDifference(['B4', 'B3']).rename('NDVI')
    ndwi = im.normalizedDifference(['B2', 'B4']).rename('NDWI')
    savi = im.expression(
              '((NIR-Red)/(NIR + Red +L)) * (1+L)', 
                 {'NIR': im.select('B4'),'Red': im.select('B3'),'L': 0.5}).rename('SAVI')
    
    
    # dowload
    if clip ==1:
        batch = clip_ls
        for i in range(0,clip_ls.size().getInfo()):

            shp = ee.Feature(batch.get(i))

            filen = fn + str(i)
            fn_ndvi = filen+'_ndvi'
            geemap.ee_export_image_to_drive(ndvi, description = fn_ndvi, folder=folder, scale=scale, region=shp.geometry())

            fn_ndwi = filen+'_ndwi'
            geemap.ee_export_image_to_drive(ndwi, description = fn_ndwi, folder=folder, scale=scale, region=shp.geometry())

            fn_savi = filen+'_savi'
            geemap.ee_export_image_to_drive(savi, description = fn_savi, folder=folder, scale=scale, region=shp.geometry())
            time.sleep(100)# help google not kill us
    else:
        batch = ee.FeatureCollection('users/templetonprojectbah/bahamas_small').toList(400)
    
        for i in step:

            shp = ee.Feature(batch.get(i))

            filen = fn + str(i)
    #         geemap.ee_export_image_to_drive(im, description = filen, folder='Grand', scale=scale, region=shp.geometry())
            fn_ndvi = filen+'_ndvi'
            geemap.ee_export_image_to_drive(ndvi, description = fn_ndvi, folder=folder, scale=scale, region=shp.geometry())

            fn_ndwi = filen+'_ndwi'
            geemap.ee_export_image_to_drive(ndwi, description = fn_ndwi, folder=folder, scale=scale, region=shp.geometry())

            fn_savi = filen+'_savi'
            geemap.ee_export_image_to_drive(savi, description = fn_savi, folder=folder, scale=scale, region=shp.geometry())
            time.sleep(100)# help google not kill us


In [ ]:
# ld = Map.draw_last_feature
# clip = temp_all.filterBounds(ld.geometry())
steps = []

# carrying it out

# 1. get batch_numbers

In [ ]:
Map = geemap.Map()
Map.addLayer(c)
Map.addLayer(s18)
Map.centerObject(c, zoom=9)
Map

In [ ]:
ld = Map.draw_last_feature
clip = batch.filterBounds(ld.geometry())
clip_ls = clip.toList(clip.size().getInfo())
step=[]

In [ ]:
clip_ls.size().getInfo()

In [ ]:
import datetime
while datetime.datetime.now().hour != 9:
    print('waiting')
    time.sleep(1200)

# 2. M18

In [ ]:
# load M18
m18 = ee.ImageCollection('projects/bahamas-339210/assets/M18')


In [ ]:
# rgb
m = download_planet_rgbOnly(ic=m18,shp=c, fn='m18',folder='M18', scale=scale, step=clip_ls, clip=1, clip_ls=clip_ls)
# time.sleep(1000)
m = download_planet_deriv(ic=m18,shp=c, fn='m18',folder='M18_d', scale=scale, step=clip_ls, clip=1, clip_ls=clip_ls)


# 3. M19

In [ ]:
m19 = ee.ImageCollection('projects/bahamas-339210/assets/M19')

In [ ]:
time.sleep(1000)

m = download_planet_rgbOnly(ic=m19,shp=c, fn='m19',folder='M19', scale=scale, step=clip_ls, clip=1, clip_ls=clip_ls)
time.sleep(1000)
m = download_planet_deriv(ic=m19,shp=c, fn='m19',folder='M19_d', scale=scale, step=clip_ls, clip=1, clip_ls=clip_ls)
# time.sleep(1000)


# 4. M20

In [ ]:
m20 = ee.ImageCollection('projects/bahamas-339210/assets/M20')

In [ ]:
m = download_planet_rgbOnly(ic=m20,shp=c, fn='m20',folder='M20', scale=scale, step=steps, clip=1, clip_ls=clip_ls)
m = download_planet_deriv(ic=m20,shp=c, fn='m20',folder='M20_d', scale=scale, step=steps, clip=1, clip_ls=clip_ls)

# 5. S18

In [ ]:
s18 = ee.ImageCollection('projects/bahamas-339210/assets/S18')

In [ ]:
# m = download_planet_rgbOnly(ic=s18,shp=c, fn='s18',folder='S18', scale=scale, step=steps, clip=1, clip_ls=clip_ls)
m = download_planet_deriv(ic=s18,shp=c, fn='s18',folder='S18_d', scale=scale, step=steps, clip=1, clip_ls=clip_ls)

# 6. S19

In [ ]:
s19 = ee.ImageCollection('projects/bahamas-339210/assets/S19')

In [ ]:
m = download_planet_rgbOnly(ic=s19,shp=c, fn='s19',folder='S19', scale=scale, step=steps, clip=1, clip_ls=clip_ls)
m = download_planet_deriv(ic=s19,shp=c, fn='s19',folder='S19_d', scale=scale, step=steps, clip=1, clip_ls=clip_ls)

# 7. S20

In [ ]:
s20 = ee.ImageCollection('projects/bahamas-339210/assets/S20')

In [ ]:
m = download_planet_rgbOnly(ic=s20,shp=c, fn='s20',folder='S20', scale=scale, step=steps, clip=1, clip_ls=clip_ls)
m = download_planet_deriv(ic=s20,shp=c, fn='s20',folder='S20_d', scale=scale, step=steps, clip=1, clip_ls=clip_ls)

----------------------